### Função para criação do arquivo final em excel

In [ ]:
import pandas as pd

def criar_excel(df, caminho='dt_pandas_to_excel.xlsx', indice=True, nome_planilha='Planilha1'):
    '''Cria um arquivo do excel formatado a partir de um data frame pandas
    
    Parameters:
        df (objeto pandas DataFrame): Data frame a ser convertido em planilha do excel
        caminho (string): Caminho do local para salvar o arquivo, com nome do arquivo e extensão
        indice (boolean): Indica se o arquivo irá ou não criar coluna para os índices do data frame
        nome_planilha (string): Nome para a aba/planilha da pasta de trabalho do excel
    
    Returns:
        Não possui retorno    
    '''
    
    # Cria um escritor pandas excel usando XlsxWriter como mecanismo e indicando o caminho como parâmetro
    escritor = pd.ExcelWriter(caminho, engine='xlsxwriter')

    # Converte o data frame em um objeto excel XlsxWriter
    df.to_excel(
        escritor,
        sheet_name=nome_planilha, # Define o nome da aba/planilha da pasta de trabalho
        startrow=1, # Pula uma linha para que seja possível inserir cabeçalho personalizado
        header=False, # Desativa o cabeçalho padrão
        index=indice # Define se irá criar coluna para índice ou não de acordo com o parâmtero informado
    )

    # Obtém a pasta de trabalho XlsxWriter e os objetos de planilha
    pasta_trabalho = escritor.book # Pasta de trabalho
    pasta_trabalho = escritor.sheets[nome_planilha] # Planilha

    # Obtém as dimensões do data frame
    (qtde_linhas, qtde_colunas) = df.shape

    # Cria uma lista de cabeçalhos das colunas para uso no add_table()
    config_colunas = []
    if indice == True: # Se Parâmetro do índice for verdadeiro, inclui o primeiro cabeçalho como 'Índice'
        config_colunas.append({'header': 'Índice'})
    for cabecalho in df.columns: # Para cada coluna no data frame, inclui o cabeçalho na lista
        config_colunas.append({'header': cabecalho})
    
    # Caso parâmetro do índice for falso, diminui uma coluna da quantidade de colunas 
    if indice == False:
        qtde_colunas -= 1
        
    # Adiciona a tabela
    pasta_trabalho.add_table(0, 0, qtde_linhas, qtde_colunas, {'columns': config_colunas})

    # Aumente a largura das colunas para melhor visualização
    pasta_trabalho.set_column(0, qtde_colunas, 25)

    # Fecha o escritor pandas excel writer e salva o arquivo excel
    escritor.save()

### Validação dos votos de evento gastronômico

In [ ]:
import pandas as pd

# Abrir planilha de votos
df_votos_brutos = pd.read_excel('votos-brutos.xlsx')

# Abrir a planilha de códigos com os estabelecimentos preenchidos
df_codigos = pd.read_excel('codigos-categorias-geral-preenchido.xlsx')

# Armazenando apenas as colunas de código, categoria e estabelecimento do data frame de códigos
df_codigos = df_codigos[['Código', 'Categoria', 'Estabelecimento']]

# Renomeando a coluna de códigos do data frame de votos, para igualar o nome com o do data frame de códigos
df_votos_brutos = df_votos_brutos.rename(columns={'Código de Validação':'Código'})
display(df_votos_brutos.info())

# Novo data frame de votos válidos recebe apenas os votos onde o código informado contiver 6 caracteres
df_votos_validos = df_votos_brutos[df_votos_brutos['Código'].str.len() == 6]
display(df_votos_validos.info())

# Novo data frame de votos inválidos recebe apenas os votos onde o código informado não contiver 6 caracteres
df_votos_invalidos = df_votos_brutos[df_votos_brutos['Código'].str.len() != 6]
display(df_votos_invalidos.info())

# Alterando todos os códigos de validação dos votos para lowercase (letras minúsculas)
# para caso alguém tenha digitado com alguma letra maiúscula por engano
df_votos_validos['Código'] = df_votos_validos['Código'].str.lower()
display(df_votos_validos.info())

# Excluir votos com códigos inseridos mais de uma vez
df_votos_validos = df_votos_validos.drop_duplicates(subset=['Código'], keep='first')
display(df_votos_validos.info())

# Atribuir o valor da categoria e do estabelecimento para cada código do data frame de votos
df_votos_validos = df_votos_validos.merge(df_codigos, on='Código')

# Ordenar as colunas recém inseridas após a coluna de códigos para melhor visualização
colunas_inicio = ['Carimbo de data/hora', 'Código', 'Categoria', 'Estabelecimento'] # Lista com as quatro primeiras colunas
todas_colunas = colunas_inicio + (df_votos_validos.columns.drop(colunas_inicio).tolist()) # Lista com todas as colunas ordenadas (as quatro primeiras e todas as colunas tirando as quatro primeiras)
df_votos_validos = df_votos_validos[todas_colunas] # Atribuindo o data frame à mesma variável com as colunas ordenadas

# Cria pasta de trabalho do excel com o data frame
criar_excel(df_votos_validos, caminho='votos-validos.xlsx', indice=False)